# classes

In [2]:
class person:
    def __init__(self, name):
        self.name=name
        
p=person('narain')

In [3]:
p.__dict__

{'name': 'narain'}

In [4]:
p.name

'narain'

In [6]:
getattr(p, 'name')

'narain'

In [7]:
setattr(p, 'name','jones')
p.name

'jones'

In [15]:
class person:
    'this is a person object'
    def __init__(self, name):
        self._name=name
    
    @property
    def name(self):
        return self._name

In [16]:
p=person('alex')
p.name

'alex'

In [17]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    # what's the property name now? --> name
    # so name has a setter callable
    @name.setter
    def name(self, value):
        self._name = value

# the property docstring is only set in the getter

In [19]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        """The Person's name."""
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

In [21]:
help(Person.name)

Help on property:

    The Person's name.



In [22]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property(doc='Write-only name property.')
    
    @name.setter
    def name(self, value):
        self._name = value

# read only properties

In [24]:
from math import pi

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @property
    def area(self):
        print('calculating area...')
        return pi * (self.radius ** 2)

In [25]:
c = Circle(1)
c.area

calculating area...


3.141592653589793

In [26]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        self._area = None
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        # if radius value is set we invalidate our cached _area value
        # we could make this more intelligent and see if the radius has actually changed
        # but keeping it simple
        self._area = None
        # we could even add validation here, like value has to be numeric, non-negative, etc
        self._radius = value
        
    @property
    def area(self):
        if self._area is None:
            # value not cached - calculate it
            print('Calculating area...')
            self._area = pi * (self.radius ** 2)
        return self._area

In [27]:
c = Circle(1)
c.area

Calculating area...


3.141592653589793

In [28]:
class WebPage:
    def __init__(self, url):
        self.url = url
        self._page = None
        self._load_time_secs = None
        self._page_size = None
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        self._url = value
        self._page = None
        # we'll lazy load the page - i.e. we wait until some property is requested
        
    @property
    def page(self):
        if self._page is None:
            self.download_page()
        return self._page
    
    @property
    def page_size(self):
        if self._page is None:
            # need to first download the page
            self.download_page()
        return self._page_size
        
    @property
    def time_elapsed(self):
        if self._page is None:
            self.download_page()
        return self._load_time_secs
            
    def download_page(self):
        self._page_size = None
        self._load_time_secs = None
        start_time = perf_counter()
        with urllib.request.urlopen(self.url) as f:
            self._page = f.read()
        end_time = perf_counter()
        
        self._page_size = len(self._page)
        self._load_time_secs = end_time - start_time

In [31]:
from time import perf_counter
import urllib

urls = [
    'https://www.google.com',
    'https://www.python.org',
    'https://www.yahoo.com'
]

for url in urls:
    page = WebPage(url)
    print(f'{url} \tsize={format(page.page_size, "_")} \telapsed={page.time_elapsed:.2f} secs')

https://www.google.com 	size=14_466 	elapsed=0.72 secs
https://www.python.org 	size=48_811 	elapsed=0.41 secs
https://www.yahoo.com 	size=463_396 	elapsed=5.72 secs


In [32]:
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def name(self):
        print('getting name property value...')
        return self._name
    
    @name.setter
    def name(self, value):
        """Person name"""
        print(f'setting name property to {value}...')
        self._name = value
    
    @name.deleter
    def name(self):
        # delete the underlying data
        print('deleting name property value...')
        del self._name

# static methods

In [1]:
class MyClass:
    def hello():
        # this IS an instance method, we just forgot to add a parameter to capture the instance
        # when this is called from an instance - so this will fail
        print('hello...')
        
    def instance_hello(arg):
        print(f'hello from {arg}')
        
    @classmethod
    def class_hello(arg):
        print(f'hello from {arg}')

In [2]:
m=MyClass()

In [3]:
MyClass.hello()

hello...


In [4]:
try:
    m.hello()
except TypeError as ex:
    print(ex)

hello() takes 0 positional arguments but 1 was given


In [5]:
m.instance_hello()

hello from <__main__.MyClass object at 0x000002273C303748>


In [6]:
try:
    MyClass.instance_hello()
except TypeError as ex:
    print(ex)

instance_hello() missing 1 required positional argument: 'arg'


As you can see, the instance method needs to be called from the instance. If we call it from the class, no argument is passed to the function, so we end up with an exception.

This is not the case with class methods - whether we call the method from the class, or the instance, that first argument will always be provided by Python, and will be the class object (not the instance).

Notice how the bindings are different:

In [7]:
MyClass.class_hello

<bound method MyClass.class_hello of <class '__main__.MyClass'>>

In [8]:
m.class_hello

<bound method MyClass.class_hello of <class '__main__.MyClass'>>

In [9]:
MyClass.class_hello()

hello from <class '__main__.MyClass'>


In [10]:
m.class_hello()

hello from <class '__main__.MyClass'>


# static methods

In [11]:
class MyClass:
    def instance_hello(self):
        print(f'Instance method bound to {self}')
        
    @classmethod
    def class_hello(cls):
        print(f'Class method bound to {cls}')
        
    @staticmethod
    def static_hello():
        print('Static method not bound to anything')

In [12]:
m = MyClass()

m.instance_hello()

Instance method bound to <__main__.MyClass object at 0x000002273CD58788>


In [13]:
MyClass.class_hello()

Class method bound to <class '__main__.MyClass'>


In [14]:
m.class_hello()

Class method bound to <class '__main__.MyClass'>


In [15]:
MyClass.static_hello

<function __main__.MyClass.static_hello()>

In [16]:
m.static_hello

<function __main__.MyClass.static_hello()>

In [17]:
MyClass.static_hello()

Static method not bound to anything


In [18]:
m.static_hello()

Static method not bound to anything


In [19]:
from datetime import datetime, timezone, timedelta

class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

So `tz` is a class attribute, and we can set it using a class method `set_timezone` - any instances will share the same `tz` value (unless we override it at the instance level)

In [20]:
Timer.set_tz(+5.30, 'MST')

In [22]:
Timer.tz

datetime.timezone(datetime.timedelta(seconds=19080), 'MST')

In [23]:
t1 = Timer()
t2 = Timer()

In [25]:
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(seconds=19080), 'MST'),
 datetime.timezone(datetime.timedelta(seconds=19080), 'MST'))

In [26]:
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

In [27]:
Timer.current_dt_utc()

datetime.datetime(2019, 12, 17, 8, 54, 51, 509820, tzinfo=datetime.timezone.utc)

In [28]:
t= Timer()
t.current_dt_utc()

datetime.datetime(2019, 12, 17, 8, 55, 22, 51659, tzinfo=datetime.timezone.utc)

Next we want a method that will return the current time based on the set time zone. Obviously the time zone is a class variable, so we'll need to access that, but we don't need any instance data, so this is a prime candidate for a class method:

In [29]:
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)
        
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)

In [30]:
Timer.current_dt_utc(), Timer.current_dt()

(datetime.datetime(2019, 12, 17, 8, 59, 48, 45984, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 12, 17, 8, 59, 48, 45984, tzinfo=datetime.timezone.utc))

In [31]:
t1=Timer()
t1.current_dt_utc(), t1.current_dt()

(datetime.datetime(2019, 12, 17, 9, 4, 59, 560767, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 12, 17, 9, 4, 59, 560767, tzinfo=datetime.timezone.utc))

In [32]:
t2 = Timer()
t2.current_dt()

datetime.datetime(2019, 12, 17, 9, 13, 47, 691446, tzinfo=datetime.timezone.utc)

In [34]:
t2.set_tz(-7, 'MST')

In [35]:
Timer.current_dt_utc(), Timer.current_dt(), t1.current_dt(), t2.current_dt()

(datetime.datetime(2019, 12, 17, 9, 14, 23, 111886, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 12, 17, 2, 14, 23, 111886, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')),
 datetime.datetime(2019, 12, 17, 2, 14, 23, 111886, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')),
 datetime.datetime(2019, 12, 17, 2, 14, 23, 111886, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')))

In [36]:
class TimerError(Exception):
    """A custom exception used for Timer class"""
    # (since """...""" is a statement, we don't need to pass)
    
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    def __init__(self):
        # use these instance variables to keep track of start/end times
        self._time_start = None
        self._time_end = None
        
    @staticmethod
    def current_dt_utc():
        """Returns non-naive current UTC"""
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)
        
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)
    
    def start(self):
        # internally we always non-naive UTC
        self._time_start = self.current_dt_utc()
        self._time_end = None
        
    def stop(self):
        if self._time_start is None:
            # cannot stop if timer was not started!
            raise TimerError('Timer must be started before it can be stopped.')
        self._time_end = self.current_dt_utc()
        
    @property
    def start_time(self):
        if self._time_start is None:
            raise TimerError('Timer has not been started.')
        # since tz is a class variable, we can just as easily access it from self
        return self._time_start.astimezone(self.tz)  
        
    @property
    def end_time(self):
        if self._time_end is None:
            raise TimerError('Timer has not been stopped.')
        return self._time_end.astimezone(self.tz)
    
    @property
    def elapsed(self):
        if self._time_start is None:
            raise TimerError('Timer must be started before an elapsed time is available')
            
        if self._time_end is None:
            # timer has not ben stopped, calculate elapsed between start and now
            elapsed_time = self.current_dt_utc() - self._time_start
        else:
            # timer has been stopped, calculate elapsed between start and end
            elapsed_time = self._time_end - self._time_start
            
        return elapsed_time.total_seconds()

In [37]:
from time import sleep

t1 = Timer()
t1.start()
sleep(2)
t1.stop()
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2019-12-17 09:24:42.408961+00:00
End time: 2019-12-17 09:24:44.411074+00:00
Elapsed: 2.002113 seconds


In [38]:
t2 = Timer()
t2.start()
sleep(3)
t2.stop()
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

Start time: 2019-12-17 09:24:57.164129+00:00
End time: 2019-12-17 09:25:00.164826+00:00
Elapsed: 3.000697 seconds


In [39]:
Timer.set_tz(-7, 'MST')

In [40]:
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2019-12-17 02:24:42.408961-07:00
End time: 2019-12-17 02:24:44.411074-07:00
Elapsed: 2.002113 seconds


In [41]:
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

Start time: 2019-12-17 02:24:57.164129-07:00
End time: 2019-12-17 02:25:00.164826-07:00
Elapsed: 3.000697 seconds


# class body scope

In [42]:
class Language:
    MAJOR = 3
    MINOR = 7
    REVISION = 4
    
    @property
    def version(self):
        return '{}.{}.{}'.format(self.MAJOR, self.MINOR, self.REVISION)
    
    @classmethod
    def cls_version(cls):
        return '{}.{}.{}'.format(cls.MAJOR, cls.MINOR, cls.REVISION)
    
    @staticmethod
    def static_version():
        return '{}.{}.{}'.format(Language.MAJOR, Language.MINOR, Language.REVISION)

In [43]:
l = Language()
l.version

'3.7.4'

In [44]:
Language.cls_version()

'3.7.4'

In [45]:
Language.static_version()

'3.7.4'

In [46]:
def full_version():
 return '{}.{}.{}'.format(Language.MAJOR, Language.MINOR, Language.REVISION)

In [48]:
full_version()

'3.7.4'

# arithmetic operators

In [50]:
from numbers import Real
from math import sqrt

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one, and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
        
    def __len__(self):
        return len(self._components)
        
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
            
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
            
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        print('__mul__ called...')
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        print('__rmul__ called...')
        # for us, multiplication is commutative, so we can leverage our existing __mul__ method
        return self * other
    
    def __iadd__(self, other):
        print('__radd__ called...')
        if self.validate_type_and_dimension(other):
            components = (x + y for x, y in zip(self.components, other.components))
            self._components = tuple(components)  # mutating our Vector object
            return self # don't forget to return the result of the operation!
        return NotImplemented
        
    def __neg__(self):
        print('__neg__ called...')
        components = (-x for x in self.components)
        return Vector(*components)
    
    def __abs__(self):
        print('__abs__ called...')
        return sqrt(sum(x ** 2 for x in self.components))

## 'Complete callables ****'